<a href="https://colab.research.google.com/github/neohack22/ML-engineering/blob/master/02a-tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converse API with tools

## Init

In [ ]:
import json
import math
import urllib
import boto3

## Define dependencies a tool error class


## Define a function to call Amazon Bedrock and return the response

We’re going to call Anthropic Claude 3 Sonnet using the converse method. We pass it a list of messages and a list of tools. We also set an output token limit and set the temperature to 0 to reduce the variability between calls (During development and testing, it can be preferable to set temperature higher for more variability in responses).
You can learn more about the Converse method [here](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html).


In [ ]:
def call_bedrock(message_list, tool_list):
    session = boto3.Session()

    bedrock = session.client(service_name="bedrock-runtime")
    if tool_list:
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
            toolConfig={"tools": tool_list},
        )
    else:
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
        )

    return response

La question Python ici concerne l'utilisation d'un client AWS Bedrock pour interagir avec un modèle de langage Claude via la fonction `call_bedrock`. Cette fonction utilise la bibliothèque `boto3` pour créer une session AWS et interagir avec le service Bedrock en envoyant des messages à un modèle de langage spécifique. Si une liste d'outils (`tool_list`) est fournie, elle est incluse dans la requête ; sinon, seule la conversation est envoyée.

Voici une explication détaillée du code et un exemple d'utilisation.

### Explication :
1. **Session AWS** : `boto3.Session()` crée une session avec les informations d'identification AWS pour appeler les services AWS.
2. **Client Bedrock** : `session.client(service_name="bedrock-runtime")` crée un client pour interagir avec le service Bedrock, utilisé pour appeler des modèles de langage.
3. **Paramètres de l'appel Bedrock** :
   - **`modelId`** : Le modèle utilisé est `anthropic.claude-3-sonnet-20240229-v1:0`, qui est une version spécifique de Claude.
   - **`messages`** : La liste de messages à envoyer au modèle (similaire à un historique de conversation).
   - **`inferenceConfig`** : Des paramètres d'inférence tels que `maxTokens` (le nombre maximum de tokens dans la réponse) et `temperature` (contrôle la créativité de la réponse).
   - **`toolConfig`** : Si `tool_list` est fourni, les outils sont inclus dans l'appel. Cela pourrait inclure des capacités supplémentaires pour le modèle, comme des outils de recherche ou de calcul.

### Exemple de code Python avec explication supplémentaire :

```python
import boto3

def call_bedrock(message_list, tool_list=None):
    # Création d'une session boto3 pour interagir avec AWS Bedrock
    session = boto3.Session()

    # Initialisation du client Bedrock pour communiquer avec le service Bedrock
    bedrock = session.client(service_name="bedrock-runtime")
    
    # Si une liste d'outils est fournie, inclure ces outils dans la requête
    if tool_list:
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
            toolConfig={"tools": tool_list},
        )
    else:
        # Sinon, envoyer seulement les messages sans outils supplémentaires
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
        )

    # Retourner la réponse du modèle Bedrock
    return response

# Exemple d'utilisation de la fonction
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "human", "content": "Quels livres Python recommandes-tu ?"},
]

# Appel sans outils
response = call_bedrock(messages)

# Affichage de la réponse
print(response)
```

### Explication des éléments clés :
1. **Paramètres `inferenceConfig`** : `maxTokens` limite le nombre de tokens dans la réponse générée par le modèle (ici 2000), et `temperature=0` produit une réponse plus déterministe.
2. **Utilisation des outils** : Si vous avez une liste d'outils supplémentaires (comme des fonctionnalités de recherche ou des calculs), vous pouvez la passer dans `tool_list` pour améliorer les capacités du modèle.

Ce code appelle un modèle de langage Claude via AWS Bedrock, en prenant en charge des configurations d'outils pour enrichir l'interaction si nécessaire.

## Add a function to handle tool use method calls

We’ll implement this function as a simple series of if/elif statements to call basic math functions or getting weather.
Note that we're deliberately skipping the tangent tool so something interesting can happen!


### Weather tool

In [ ]:
def get_weather(city: str):
    encoded_city = urllib.parse.quote(city)
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json"
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data["results"]:
            return {"error": "City not found"}

        lat = location_data["results"][0]["latitude"]
        lon = location_data["results"][0]["longitude"]

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto"

    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data["current"]
    daily = weather_data["daily"]

    weather_codes = {
        0: "Clear sky",
        1: "Mainly clear",
        2: "Partly cloudy",
        3: "Overcast",
        45: "Fog",
        48: "Depositing rime fog",
        51: "Light drizzle",
        53: "Moderate drizzle",
        55: "Dense drizzle",
        61: "Slight rain",
        63: "Moderate rain",
        65: "Heavy rain",
        71: "Slight snow fall",
        73: "Moderate snow fall",
        75: "Heavy snow fall",
        77: "Snow grains",
        80: "Slight rain showers",
        81: "Moderate rain showers",
        82: "Violent rain showers",
        85: "Slight snow showers",
        86: "Heavy snow showers",
        95: "Thunderstorm",
        96: "Thunderstorm with slight hail",
        99: "Thunderstorm with heavy hail",
    }
    response_core = {
        "temperature": current["temperature_2m"],
        "condition": weather_codes.get(current["weather_code"], "Unknown"),
        "humidity": current["relative_humidity_2m"],
        "wind_speed": current["wind_speed_10m"],
        "forecast_max": daily["temperature_2m_max"][0],
        "forecast_min": daily["temperature_2m_min"][0],
        "forecast_condition": weather_codes.get(daily["weather_code"][0], "Unknown"),
    }
    return response_core

La question Python ici concerne la fonction `get_weather`, qui permet de récupérer les informations météorologiques d'une ville donnée. Cette fonction utilise l'API Open Meteo pour obtenir les coordonnées géographiques de la ville, puis elle interroge l'API pour obtenir les données météorologiques actuelles et les prévisions. Voici une explication détaillée de cette fonction ainsi qu'un exemple d'utilisation.

### Explication :
1. **Encodage du nom de la ville** : `encoded_city = urllib.parse.quote(city)` encode le nom de la ville pour le rendre sûr pour l'inclusion dans une URL.
2. **Construction de l'URL de géocodage** : La première API est utilisée pour obtenir les coordonnées géographiques (latitude et longitude) de la ville.
3. **Gestion de la réponse** : Si aucune ville n'est trouvée (`if not location_data["results"]`), la fonction retourne un message d'erreur.
4. **Appel de l'API météo** : Une fois que les coordonnées sont obtenues, une deuxième requête est effectuée pour récupérer les données météorologiques actuelles et les prévisions.
5. **Codes météo** : Un dictionnaire `weather_codes` est utilisé pour traduire les codes de condition météo en descriptions lisibles.
6. **Réponse formatée** : La fonction retourne un dictionnaire contenant la température actuelle, la condition météorologique, l'humidité, la vitesse du vent et les prévisions de température maximale et minimale.

### Exemple de code Python :

```python
import urllib.parse
import urllib.request
import json

def get_weather(city: str):
    # Encoder le nom de la ville pour l'URL
    encoded_city = urllib.parse.quote(city)
    # URL pour l'API de géocodage
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json"
    
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data["results"]:
            return {"error": "City not found"}

        lat = location_data["results"][0]["latitude"]
        lon = location_data["results"][0]["longitude"]

    # URL pour l'API météo
    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto"

    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data["current"]
    daily = weather_data["daily"]

    # Dictionnaire des codes météo
    weather_codes = {
        0: "Clear sky",
        1: "Mainly clear",
        2: "Partly cloudy",
        3: "Overcast",
        45: "Fog",
        48: "Depositing rime fog",
        51: "Light drizzle",
        53: "Moderate drizzle",
        55: "Dense drizzle",
        61: "Slight rain",
        63: "Moderate rain",
        65: "Heavy rain",
        71: "Slight snow fall",
        73: "Moderate snow fall",
        75: "Heavy snow fall",
        77: "Snow grains",
        80: "Slight rain showers",
        81: "Moderate rain showers",
        82: "Violent rain showers",
        85: "Slight snow showers",
        86: "Heavy snow showers",
        95: "Thunderstorm",
        96: "Thunderstorm with slight hail",
        99: "Thunderstorm with heavy hail",
    }

    # Réponse formatée
    response_core = {
        "temperature": current["temperature_2m"],
        "condition": weather_codes.get(current["weather_code"], "Unknown"),
        "humidity": current["relative_humidity_2m"],
        "wind_speed": current["wind_speed_10m"],
        "forecast_max": daily["temperature_2m_max"][0],
        "forecast_min": daily["temperature_2m_min"][0],
        "forecast_condition": weather_codes.get(daily["weather_code"][0], "Unknown"),
    }
    
    return response_core

# Exemple d'utilisation de la fonction
weather_info = get_weather("Paris")
print(weather_info)
```

### Explication supplémentaire :
- **Imports** : `urllib.parse`, `urllib.request` et `json` sont importés pour gérer les requêtes HTTP et le traitement JSON.
- **Gestion des erreurs** : La fonction gère les cas où la ville n'est pas trouvée, ce qui améliore la robustesse.
- **Utilisation de la fonction** : L'exemple montre comment appeler `get_weather` avec "Paris" et afficher les informations météo.

Cette fonction fournit une façon simple et efficace d'obtenir des données météorologiques à partir d'une API publique.

In [ ]:
get_weather("Paris")

{'temperature': 22.6,
 'condition': 'Clear sky',
 'humidity': 66,
 'wind_speed': 14.0,
 'forecast_max': 24.6,
 'forecast_min': 15.5,
 'forecast_condition': 'Moderate rain'}

### Defining the tools



We’re using a custom ToolError class to handle some of the potential things that can go wrong with tool use.

In [ ]:
class ToolError(Exception):
    pass

In [ ]:
def get_tool_result(tool_use_block):

    tool_use_name = tool_use_block["name"]

    print(f"Using tool {tool_use_name}")

    # Note: We're deliberately excluding tangent so something magical can happen
    if tool_use_name == "cosine":
        return math.cos(tool_use_block["input"]["x"])
    elif tool_use_name == "sine":
        return math.sin(tool_use_block["input"]["x"])
    elif tool_use_name == "divide_numbers":
        return tool_use_block["input"]["x"] / tool_use_block["input"]["y"]
    elif tool_use_name == "get_weather":
        return get_weather(tool_use_block["input"]["city"])
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")

La question Python ici concerne la fonction `get_tool_result`, qui permet d'exécuter différentes opérations en fonction du nom de l'outil fourni dans un bloc d'utilisation d'outil. Cette fonction utilise des conditions pour déterminer quelle opération exécuter et renvoie le résultat approprié. Voici une explication détaillée de cette fonction.

### Explication :
1. **Extraction du nom de l'outil** : La première ligne récupère le nom de l'outil à partir du bloc d'utilisation (`tool_use_block`).
2. **Affichage de l'outil utilisé** : `print(f"Using tool {tool_use_name}")` affiche le nom de l'outil utilisé pour le débogage.
3. **Conditions pour chaque outil** :
   - **Cosinus** : Si l'outil est "cosine", il appelle `math.cos` avec l'argument `x` provenant de `tool_use_block["input"]`.
   - **Sinus** : Si l'outil est "sine", il appelle `math.sin` de la même manière.
   - **Division** : Si l'outil est "divide_numbers", il effectue une division avec les valeurs `x` et `y`.
   - **Météo** : Si l'outil est "get_weather", il appelle la fonction `get_weather` avec le nom de la ville.
4. **Gestion des erreurs** : Si le nom de l'outil n'est pas reconnu, une exception `ToolError` est levée avec un message d'erreur.

### Exemple de code Python :

```python
import math

class ToolError(Exception):
    pass

def get_tool_result(tool_use_block):
    # Extraction du nom de l'outil à partir du bloc d'utilisation
    tool_use_name = tool_use_block["name"]
    
    print(f"Using tool {tool_use_name}")

    # Exécuter l'opération correspondante en fonction du nom de l'outil
    if tool_use_name == "cosine":
        return math.cos(tool_use_block["input"]["x"])
    elif tool_use_name == "sine":
        return math.sin(tool_use_block["input"]["x"])
    elif tool_use_name == "divide_numbers":
        return tool_use_block["input"]["x"] / tool_use_block["input"]["y"]
    elif tool_use_name == "get_weather":
        return get_weather(tool_use_block["input"]["city"])
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")

# Exemple d'utilisation de la fonction
tool_input = {
    "name": "sine",
    "input": {
        "x": 1.5708  # Environ π/2
    }
}

result = get_tool_result(tool_input)
print(result)  # Affiche le résultat du sinus
```

### Explication des éléments clés :
- **Classe d'exception `ToolError`** : Cette classe permet de lever des erreurs spécifiques lorsqu'un outil invalide est demandé.
- **Structure de `tool_use_block`** : La fonction attend un dictionnaire contenant le nom de l'outil et les entrées nécessaires pour effectuer le calcul.
- **Utilisation de la fonction** : L'exemple montre comment appeler `get_tool_result` avec un bloc d'outil pour calculer le sinus de π/2, ce qui devrait retourner 1.

Cette fonction est utile pour encapsuler diverses opérations mathématiques et de services (comme la récupération des données météorologiques) dans un format uniforme et extensible.

## Add a function to handle LLM responses and determine if a follow-up tool call is needed
The LLM may return a combination of text and tool use content blocks in its response. We’ll look for tooUse content blocks, attempt to run the requested tools, and return a message with a toolResult block if a tool was used.



In [ ]:
def handle_response(response_message):

    response_content_blocks = response_message["content"]

    follow_up_content_blocks = []

    for content_block in response_content_blocks:
        if "toolUse" in content_block:
            tool_use_block = content_block["toolUse"]

            try:
                tool_result_value = get_tool_result(tool_use_block)

                if tool_result_value is not None:
                    follow_up_content_blocks.append(
                        {
                            "toolResult": {
                                "toolUseId": tool_use_block["toolUseId"],
                                "content": [{"json": {"result": tool_result_value}}],
                            }
                        }
                    )

            except ToolError as e:
                follow_up_content_blocks.append(
                    {
                        "toolResult": {
                            "toolUseId": tool_use_block["toolUseId"],
                            "content": [{"text": repr(e)}],
                            "status": "error",
                        }
                    }
                )

    if len(follow_up_content_blocks) > 0:

        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }

        return follow_up_message
    else:
        return None

La question Python ici concerne la fonction `handle_response`, qui traite un message de réponse et gère l'utilisation des outils en fonction des blocs de contenu fournis. Voici une explication détaillée de cette fonction.

### Explication :
1. **Extraction du contenu de la réponse** : La fonction commence par extraire les blocs de contenu de la réponse (`response_message["content"]`).
2. **Liste pour les réponses de suivi** : Une liste `follow_up_content_blocks` est initialisée pour stocker les résultats ou les erreurs.
3. **Boucle sur les blocs de contenu** : Pour chaque bloc de contenu dans `response_content_blocks`, la fonction vérifie s'il contient un `toolUse`.
   - Si un bloc `toolUse` est trouvé, il est traité avec la fonction `get_tool_result`.
   - **Gestion des erreurs** : Si une exception `ToolError` est levée lors de l'exécution de `get_tool_result`, l'erreur est capturée et ajoutée à la liste des blocs de contenu de suivi.
4. **Création d'un message de suivi** : Si des résultats ou des erreurs ont été collectés, un message de suivi est créé et renvoyé. Sinon, la fonction retourne `None`.

### Exemple de code Python :

```python
class ToolError(Exception):
    pass

def handle_response(response_message):
    # Extraire les blocs de contenu de la réponse
    response_content_blocks = response_message["content"]
    follow_up_content_blocks = []

    # Traiter chaque bloc de contenu
    for content_block in response_content_blocks:
        if "toolUse" in content_block:
            tool_use_block = content_block["toolUse"]

            try:
                # Obtenir le résultat de l'outil
                tool_result_value = get_tool_result(tool_use_block)

                if tool_result_value is not None:
                    follow_up_content_blocks.append(
                        {
                            "toolResult": {
                                "toolUseId": tool_use_block["toolUseId"],
                                "content": [{"json": {"result": tool_result_value}}],
                            }
                        }
                    )

            except ToolError as e:
                follow_up_content_blocks.append(
                    {
                        "toolResult": {
                            "toolUseId": tool_use_block["toolUseId"],
                            "content": [{"text": repr(e)}],
                            "status": "error",
                        }
                    }
                )

    # Créer un message de suivi si des résultats sont disponibles
    if len(follow_up_content_blocks) > 0:
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        return follow_up_message
    else:
        return None

# Exemple d'utilisation de la fonction
response_message = {
    "content": [
        {
            "toolUse": {
                "toolUseId": 1,
                "name": "sine",
                "input": {"x": 1.5708}  # Environ π/2
            }
        }
    ]
}

# Appeler la fonction et afficher le résultat
result = handle_response(response_message)
print(result)
```

### Explication des éléments clés :
- **Classe d'exception `ToolError`** : Cette classe permet de gérer les erreurs spécifiques liées aux outils utilisés.
- **Structure de `response_message`** : La fonction attend un dictionnaire contenant les blocs de contenu avec des `toolUse`.
- **Utilisation de la fonction** : L'exemple montre comment appeler `handle_response` avec un message de réponse qui demande à utiliser l'outil "sine" pour calculer le sinus de π/2.

Cette fonction est essentielle pour gérer les interactions dynamiques avec différents outils en fonction des messages reçus et pour structurer la réponse à renvoyer à l'utilisateur. Elle assure également une gestion d'erreurs robuste, ce qui est crucial dans des systèmes basés sur des API.

## Add a function to run the request/response loop
This function will run a request / response loop until either the LLM stops requesting tool use or a maximum number of loops have run.


In [ ]:
def run_loop(prompt, tool_list):
    MAX_LOOPS = 6
    loop_count = 0
    continue_loop = True

    message_list = [{"role": "user", "content": [{"text": prompt}]}]

    while continue_loop:
        response = call_bedrock(message_list, tool_list)

        response_message = response["output"]["message"]
        message_list.append(response_message)

        loop_count = loop_count + 1

        if loop_count >= MAX_LOOPS:
            print(f"Hit loop limit: {loop_count}")
            break

        follow_up_message = handle_response(response_message)

        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False
        else:
            message_list.append(follow_up_message)

    return message_list

La fonction `run_loop` est conçue pour exécuter une boucle qui envoie un message à un service (ici représenté par `call_bedrock`) et gère les réponses potentielles jusqu'à un certain nombre de répétitions ou jusqu'à ce qu'il n'y ait plus de travail à effectuer. Voici une explication détaillée de son fonctionnement.

### Explication de la fonction `run_loop` :

1. **Initialisation** :
   - `MAX_LOOPS` est défini à 6, ce qui signifie que la boucle peut s'exécuter un maximum de 6 fois.
   - `loop_count` est initialisé à 0 pour suivre le nombre de boucles effectuées.
   - `continue_loop` est un indicateur qui contrôle l'exécution de la boucle.
   - `message_list` est initialisé avec le message utilisateur.

2. **Boucle principale** :
   - La boucle `while` continue tant que `continue_loop` est vrai.
   - À chaque itération, la fonction appelle `call_bedrock` avec la liste des messages et la liste des outils (`tool_list`).
   - La réponse est ensuite extraite et ajoutée à `message_list`.
   - `loop_count` est incrémenté à chaque itération.

3. **Limite de boucle** :
   - Si `loop_count` atteint `MAX_LOOPS`, un message est imprimé et la boucle se termine.

4. **Gestion des réponses** :
   - La fonction appelle `handle_response` avec le message de réponse.
   - Si `handle_response` retourne `None`, cela signifie qu'il n'y a plus de travail à faire, et la boucle se termine.
   - Sinon, le message de suivi est ajouté à `message_list`.

5. **Retour** :
   - À la fin, `message_list`, qui contient tous les messages échangés, est renvoyé.

### Exemple de code Python :

Voici la fonction `run_loop` intégrée dans un exemple :

```python
def run_loop(prompt, tool_list):
    MAX_LOOPS = 6
    loop_count = 0
    continue_loop = True

    message_list = [{"role": "user", "content": [{"text": prompt}]}]

    while continue_loop:
        response = call_bedrock(message_list, tool_list)

        response_message = response["output"]["message"]
        message_list.append(response_message)

        loop_count += 1

        if loop_count >= MAX_LOOPS:
            print(f"Hit loop limit: {loop_count}")
            break

        follow_up_message = handle_response(response_message)

        if follow_up_message is None:
            # Pas de travail restant à faire, retournez la réponse finale à l'utilisateur
            continue_loop = False
        else:
            message_list.append(follow_up_message)

    return message_list

# Exemple d'utilisation de la fonction
prompt = "Quelle est la météo à Paris aujourd'hui ?"
tool_list = ["get_weather"]  # Exemple d'outils à utiliser
result = run_loop(prompt, tool_list)
print(result)
```

### Explications supplémentaires :
- **`call_bedrock`** : Cette fonction est supposée interagir avec un service (par exemple, un modèle de langage) pour obtenir des réponses basées sur les messages envoyés.
- **`handle_response`** : Cette fonction traite les réponses reçues et renvoie des messages de suivi si nécessaire.
- **Gestion des boucles** : L'utilisation de `MAX_LOOPS` permet d'éviter des boucles infinies et de garantir que le programme reste réactif.

Cette structure permet de gérer les interactions avec un service de manière organisée, en s'assurant que toutes les réponses sont correctement traitées et que l'utilisateur reçoit la réponse finale appropriée.

## Define the tools to use
We’re defining four tools for basic trigonometry functions, a division function and get weather.To deep dive into tool definition format, check [here](https://community.aws/content/2hWA16FSt2bIzKs0Z1fgJBwu589/generating-json-with-the-amazon-bedrock-converse-api).


In [ ]:
tools = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "sine",
            "description": "Calculate the sine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "tangent",
            "description": "Calculate the tangent of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "divide_numbers",
            "description": "Divide x by y.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {"type": "number", "description": "The numerator."},
                        "y": {"type": "number", "description": "The denominator."},
                    },
                    "required": ["x", "y"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "get_weather",
            "description": "Get the weather for a city.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "city": {"type": "string", "description": "The city to get the weather for."},
                    },
                    "required": ["city"],
                }
            },
        }
    },
]

## Pass a prompt to start the loop


### Tangent
We’re asking Anthropic Claude to calculate the tangent of 7. We’re expecting a response with the calculated value.


But...

In [ ]:
messages = run_loop("What is the tangent of 7 ?", [])

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=2))


MESSAGES:

[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the tangent of 7 ?"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "The tangent function (tan) is a trigonometric function that gives the ratio of the opposite side to the adjacent side of a right-angled triangle.\n\nHowever, the tangent is only defined for angles between -90\u00b0 and 90\u00b0 (excluding -90\u00b0 and 90\u00b0). This is because for angles outside this range, the triangle would have an undefined opposite or adjacent side.\n\nThe value 7 by itself does not represent an angle in radians or degrees. So the tangent of 7 is undefined or meaningless in the context of trigonometry.\n\nIf you meant to ask for the tangent of an angle measured in radians or degrees, you would need to provide that angle value instead of just the number 7."
      }
    ]
  }
]


Le code que vous avez partagé définit une liste d'outils, chacun spécifiant un nom, une description et un schéma d'entrée pour des fonctions mathématiques et des services météorologiques. Voici une explication détaillée des différentes parties de ce code.

### Explication du code

1. **Définition des outils** :
   - Chaque outil est un dictionnaire contenant des spécifications de l'outil (`toolSpec`).
   - `toolSpec` inclut un `name`, une `description` et un `inputSchema` qui décrit les entrées nécessaires.

2. **Outils mathématiques** :
   - Les outils `cosine`, `sine` et `tangent` calculent respectivement le cosinus, le sinus et la tangente d'un nombre donné `x`.
   - Chaque outil a un schéma d'entrée qui exige que l'entrée soit un objet JSON contenant une propriété `x` de type `number`.

3. **Outil de division** :
   - L'outil `divide_numbers` divise un nombre `x` par un nombre `y`.
   - Il nécessite à la fois `x` et `y` dans son schéma d'entrée.

4. **Outil météo** :
   - L'outil `get_weather` permet d'obtenir la météo d'une ville donnée.
   - Il exige une propriété `city` de type `string`.

### Exemple de code

Voici le code structuré en utilisant la syntaxe markdown :

```python
tools = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "sine",
            "description": "Calculate the sine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "tangent",
            "description": "Calculate the tangent of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "divide_numbers",
            "description": "Divide x by y.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {"type": "number", "description": "The numerator."},
                        "y": {"type": "number", "description": "The denominator."},
                    },
                    "required": ["x", "y"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "get_weather",
            "description": "Get the weather for a city.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "city": {"type": "string", "description": "The city to get the weather for."},
                    },
                    "required": ["city"],
                }
            },
        }
    },
]
```

### Points importants :
- **Validation des entrées** : Le schéma d'entrée assure que les bonnes données sont fournies pour chaque fonction.
- **Extensibilité** : Cette structure permet d'ajouter facilement de nouveaux outils en suivant le même format.

Ce code est utile dans le cadre d'une application qui pourrait avoir besoin de fonctions mathématiques et de services météo, offrant ainsi une interface claire pour interagir avec ces outils. Si vous avez d'autres questions ou avez besoin d'explications supplémentaires sur une partie spécifique, n'hésitez pas à demander !

What if we we use tools ?

In [ ]:
messages = run_loop("What is the tangent of 7 ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=2))

Using tool tangent
Using tool sine
Using tool cosine
Using tool divide_numbers

MESSAGES:

[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the tangent of 7 ?"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "To calculate the tangent of 7, we can use the \"tangent\" tool:"
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_rm1njQ83SSCXl6pvy4SXgw",
          "name": "tangent",
          "input": {
            "x": 7
          }
        }
      }
    ]
  },
  {
    "role": "user",
    "content": [
      {
        "toolResult": {
          "toolUseId": "tooluse_rm1njQ83SSCXl6pvy4SXgw",
          "content": [
            {
              "text": "ToolError('Invalid function name: tangent')"
            }
          ],
          "status": "error"
        }
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "Oops, it seems the \"tangent\" tool is not available in this en

### Weather

We can ask Anthropic Claude to get the weather in Paris. We’re expecting a response with the current weather in Paris.

But...

In [ ]:
messages = run_loop("What is Paris weather ?", [])

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is Paris weather ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Unfortunately, I don't have up-to-the-minute weather data for Paris. The weather can vary quite a bit in Paris depending on the time of year. Here are some general points about the typical weather in Paris:\n\n- Spring (March-May) - Mild temperatures, with highs around 15-20\u00b0C. Spring can be rainy at times.\n\n- Summer (June-August) - Warm to hot, with average highs around 24-27\u00b0C. Summer brings more sunshine but can also have occasional thunderstorms.\n\n- Fall (September-November) - Cool temperatures with highs of 15-20\u00b0C. Fall tends to be rainier than summer.\n\n- Winter (December-February) - Quite cold, with average highs around 7-9\u00b0C and lows near freezing. Winter brings clouds, rain and occasional snow flurrie

What if we we use tools ?

In [ ]:
messages = run_loop("What is Paris weather ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))

Using tool get_weather

MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is Paris weather ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Okay, let me get the weather for Paris using the available tool:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_GKlTG6xnTsGnZYtzfN_-cg",
                    "name": "get_weather",
                    "input": {
                        "city": "Paris"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_GKlTG6xnTsGnZYtzfN_-cg",
                    "content": [
                        {
                            "json": {
                                "result": {
                                    "temp

### Not needing tool question, but still using tools


In [ ]:
messages = run_loop("Who is Barack Obama ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "Who is Barack Obama ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Barack Obama is an American politician who served as the 44th president of the United States from 2009 to 2017. Some key facts about him:\n\n- He was born on August 4, 1961 in Honolulu, Hawaii. He was the first African American president.\n\n- Before becoming president, he was a U.S. Senator representing Illinois from 2005 to 2008. \n\n- His presidential campaign calling for hope and change resonated with many Americans. He was elected in 2008 defeating John McCain and was re-elected in 2012 defeating Mitt Romney.\n\n- Major achievements as president include the Affordable Care Act (Obamacare) to reform healthcare, the economic stimulus package to address the Great Recession, the repeal of Don't Ask Don't Tell allowing LGBT people to ser